In [9]:
import tensorflow as tf
import re
import numpy as np

In [10]:

def split_words(line):
    if ">" in line and "<" in line:
        pattern = r'>(.*?)<'
        words = re.findall(pattern, line)
    else:
        words_quot = re.findall(r'"(.*?)"', line)
        if words_quot:
            return words_quot

        words = re.split(r'\s+', line)

    words = [word.strip() for word in words if word.strip()]

    return words

In [15]:
model = tf.keras.models.load_model('model')
data = []
with open('../result/0.txt', 'r', encoding='utf-8') as file:
    for line in file:
                words = split_words(line)
                data += words
                print(words)


['Przejdź do&nbsp;głównej treści', 'Znajdź wolny termin', 'spośród', '6 910 833', 'dostępnych na&nbsp;najbliższy rok', 'Znajdź wolny termin', 'spośród', '6 910 833', 'dostępnych na&nbsp;najbliższy rok']
['Poprzednia', 'Następna']


In [16]:
# Tokenizacja
def ngram_tokenizer(text, n, max_ngram_length):
    ngrams = [text[i:i+n] for i in range(len(text) - n + 1)]
    padded_ngrams = [ord(char) / 255 for ngram in ngrams for char in ngram]
    padded_ngrams += [0] * (max_ngram_length - len(padded_ngrams))
    return padded_ngrams


In [17]:
# Funkcja do tokenizacji 
def tokenize_text(text):
    max_word_length = 200
    max_ngram_length = 4 * max_word_length  

    test_xs_tensor = [ngram_tokenizer(word, n=3, max_ngram_length=max_ngram_length)[:max_word_length] for word in text]
    test_stacked_tensor = np.array(test_xs_tensor)
    return test_stacked_tensor

tokens = tokenize_text(data)


In [18]:
count_specializations = 0
max_word_length = 200 
max_ngram_length = 4 * max_word_length 

test_xs_tensor = [ngram_tokenizer(word, n=3, max_ngram_length=max_ngram_length)[:max_word_length] for word in data]
test_stacked_tensor = np.array(test_xs_tensor)

predictions = model.predict(test_stacked_tensor)


count_specializations = sum(1 for prediction in predictions if prediction >= 0.4)

for word, prediction in zip(data, predictions):
    is_medical_specialty = "Tak" if prediction >= 0.4 else "Nie"
    print(f"{word},  {prediction} : {is_medical_specialty}")

print(f"Ilość predykcji powyżej 0.4: {count_specializations}")

1/1 [==============================] - 0s 120ms/step
Przejdź do&nbsp;głównej treści,  [1.] : Tak
Znajdź wolny termin,  [0.9998752] : Tak
spośród,  [0.06902856] : Nie
6 910 833,  [0.01037727] : Nie
dostępnych na&nbsp;najbliższy rok,  [1.] : Tak
Znajdź wolny termin,  [0.9998752] : Tak
spośród,  [0.06902856] : Nie
6 910 833,  [0.01037727] : Nie
dostępnych na&nbsp;najbliższy rok,  [1.] : Tak
Poprzednia,  [0.70842904] : Tak
Następna,  [0.7162374] : Tak
Ilość predykcji powyżej 0.4: 7
